In [14]:
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.layers import Lambda,Conv2D,Dense,Input,Flatten,Conv2DTranspose,LeakyReLU,Reshape
from keras.models import Model
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint,EarlyStopping
from keras.datasets import mnist

In [2]:
(x_train,_),(x_test,_) = mnist.load_data()


In [3]:
size = x_train.shape[1]
x_train = np.reshape(x_train,[-1,size,size,1])
x_test = np.reshape(x_test,[-1,size,size,1])

In [4]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [5]:
# Network params
layers = [32,64,128]
kernel_size = 3
batch_size = 32
input_shape = (size,size,1)
latent_dim = 4


In [6]:
# Encoder Network
input_layer = Input(shape=input_shape,name='input_encoder_layer')
x = input_layer

for li in layers:
    conv_layer = Conv2D(filters=li,
               kernel_size=kernel_size,
               strides = 2,
               padding='same',
               name = 'encoder_conv_2d_layer'+str(li))
    x = conv_layer(x)
    x = LeakyReLU()(x)
    
shape = K.int_shape(x)

x = Flatten()(x)
mu = Dense(units = latent_dim,name='mu')(x)

log_var = Dense(units=latent_dim,name='log_var')(x)

encoder = Model(input_layer,(mu,log_var),name='Encoder_mu_logvar')

In [8]:
def sampling(args):
    mu,log_var = args
    epsilon = K.random_normal(shape=K.shape(mu),mean=0,stddev=1)
    z = mu + epsilon * K.exp(log_var/2)

    return z

In [9]:
encoder_output_layer = Lambda(sampling)([mu,log_var])

encoder_model = Model(input_layer,encoder_output_layer)

In [10]:
encoder_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_encoder_layer (InputLaye  [(None, 28, 28, 1)]  0          []                               
 r)                                                                                               
                                                                                                  
 encoder_conv_2d_layer32 (Conv2  (None, 14, 14, 32)  320         ['input_encoder_layer[0][0]']    
 D)                                                                                               
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 14, 14, 32)   0           ['encoder_conv_2d_layer32[0][0]']
                                                                                              

In [16]:
# Decoder network
dense_shape = shape[1]*shape[2]*shape[3]

decoder_input_layer = Input(shape=(latent_dim,),name='decoder_input_layer')

x = Dense(dense_shape)(decoder_input_layer)
x = Reshape((shape[1],shape[2],shape[3]))(x)

for n_units in layers[::-1]:
    conv_t_layer = Conv2DTranspose(filters= n_units,
                                   kernel_size=kernel_size,
                                   strides=2,
                                   padding='same')
    x = conv_t_layer(x)
    x = LeakyReLU()(x)
    
    decoder_output_layer = Conv2DTranspose(filters = 1,kernel_size=3,padding='same',activation='sigmoid')(x)

    decoder_model = Model(decoder_input_layer,decoder_output_layer)

In [17]:
decoder_model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input_layer (InputL  [(None, 16)]             0         
 ayer)                                                           
                                                                 
 dense_1 (Dense)             (None, 2048)              34816     
                                                                 
 reshape_1 (Reshape)         (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose_2 (Conv2DT  (None, 8, 8, 128)        147584    
 ranspose)                                                       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 16, 16, 64)       7379

In [18]:
VAE = Model(input_layer,decoder_model(encoder_model(input_layer)))

In [21]:
VAE.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_encoder_layer (InputL  [(None, 28, 28, 1)]      0         
 ayer)                                                           
                                                                 
 model (Functional)          (None, 16)                158240    
                                                                 
 model_5 (Functional)        (None, 32, 32, 1)         274945    
                                                                 
Total params: 433,185
Trainable params: 433,185
Non-trainable params: 0
_________________________________________________________________
